# Author: Dinesh Sadhwani

### **Objective:** 
A large media conglomerate has hired us to figure out which JCDecaux bike stations across the cities of Lyon, Dublin, Bruxelles, and Toyama have the most active bike ridership. The media company aims to setup electronic ad panels and other media near busy bike stations to get impressions for their clients. We perform Data Analysis to answer the following questions:
1. If we compare the 4 cities, which city has the most active bike ridership and which city has the least active one?
2. Which city has the maximum total stands capacity across all its stations? Total capacity is sum of main stand capacity and overflow stand capacity.
3. Pre-christmas, on a daily basis, which are the top 10 stations having the most active ridership?
4. Pre-christmas, on an hourly basis, which are the top 10 stations having the most active ridership?
5. Do the cities in the lists from 3 and 4 change significantly post-christmas?
6. Of the cities in the dataset that have stations with both mechanical and electric bikes, are there any cities having strong preference for either mechanical or electric bikes?

#### **About the Dataset:** 
The dataset describes the status of each bike rental stand in the cities of Lyon (France), Bruxelles (Belgium), Dublin (Ireland), and Toyama (Japan). The cities are indicated by the contractName column. The columns contractName and number uniquely identify each bike stand. The lastUpdate attribute specifies the time when the status of the bike rental station was last updated. Although the JCDecaux API server updates data every minute, for the stations that do not see a lot of activity, there is a significant difference between any 2 consecutive values of lastUpdate. On the contrary, the stations with a lot of activity have a few minutes of difference between any 2 consecutive values of lastUpdate. We will mainly be working with totalStands_availabilities.bikes (number of all bikes that are available for renting), totalStands_availabilities.stands (number of empty stands at a station), totalStands_availabilities.mechanicalBikes (of the available total bikes, how many are mechanical), and totalStands_availabilities.electricalBikes (of the available total bikes, how many are electric)

In [1]:
# importing all the required python libraries

import pandas as pd
import glob

pd.set_option('display.max_rows', 1000)

### Importing and Pre-processing Data

The bikes_data_df will combine and hold both pre- and post- christmas data

In [2]:
bikes_data_df = pd.concat([pd.read_csv(f) for f in glob.glob("./data/" + "*.csv")], ignore_index=True)

Pandas info() function tells us that only the address column has missing values

In [3]:
bikes_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005859 entries, 0 to 1005858
Data columns (total 26 columns):
 #   Column                                                      Non-Null Count    Dtype  
---  ------                                                      --------------    -----  
 0   number                                                      1005859 non-null  int64  
 1   contractName                                                1005859 non-null  object 
 2   name                                                        1005859 non-null  object 
 3   address                                                     909141 non-null   object 
 4   position                                                    1005859 non-null  object 
 5   banking                                                     1005859 non-null  bool   
 6   bonus                                                       1005859 non-null  bool   
 7   status                                                      100

The lastUpdate column needs to be converted to a pandas datetime object and we need to remove its timezone awareness 

In [4]:
# converting lastUpdate to datetime

bikes_data_df.lastUpdate = pd.to_datetime(bikes_data_df.lastUpdate)

# removing timezone awareness

bikes_data_df.lastUpdate = bikes_data_df.lastUpdate.dt.tz_convert(None)

Next we extract the day, month, and hour from the lastUpdate column. These columns will be needed for our analysis later

In [5]:
# extract day

bikes_data_df["last_update_day"] = bikes_data_df['lastUpdate'].dt.day

# extract month

bikes_data_df["last_update_month"] = bikes_data_df['lastUpdate'].dt.month

# extract hour of the day

bikes_data_df["last_update_hour"] = bikes_data_df['lastUpdate'].dt.hour

Grouping by day and month, we notice that there are certain rows that have stale data, i.e., data that does not fall in our analytics timeframe

In [6]:
bikes_data_df.groupby(["last_update_day", "last_update_month"]).size()

last_update_day  last_update_month
18               11                        2
19               12                    26900
20               12                   170093
21               12                   169656
22               11                        2
                 12                   164545
25               12                        1
26               12                   126916
27               12                   146971
28               12                   151912
29               12                    48859
30               9                         2
dtype: int64

Let's have a look at these rows with outdated data

In [7]:
bikes_data_df[bikes_data_df["last_update_month"] < 12]

,number,contractName,name,address,position,banking,bonus,status,lastUpdate,connected,...,mainStands_capacity,mainStands_availabilities.bikes,mainStands_availabilities.stands,mainStands_availabilities.mechanicalBikes,mainStands_availabilities.electricalBikes,mainStands_availabilities.electricalInternalBatteryBikes,mainStands_availabilities.electricalRemovableBatteryBikes,last_update_day,last_update_month,last_update_hour
511,507,dublin,ORIEL STREET TEST TERMINAL,"JCDecaux Ireland, 52 Oriel Street Lower, Dublin 1","{'latitude': 53.35463, 'longitude': -6.242615}",False,False,OPEN,2021-11-18 07:11:16,True,...,1,1,0,1,0,0,0,18,11,7
622,208,bruxelles,208 - MELBA,MELBA - AV NELLIE MELBA (FACE 1 a 9) / NELLIE ...,"{'latitude': 50.832565, 'longitude': 4.294311}",False,False,CLOSED,2021-11-22 08:35:26,False,...,25,0,0,0,0,0,0,22,11,8
770,286,bruxelles,286 - BENS,BENS - CHEE D'ALSEMBERG DEVANT LE 548 / ALSEMB...,"{'latitude': 50.807811, 'longitude': 4.336789}",False,False,CLOSED,2021-09-30 08:37:53,False,...,20,0,0,0,0,0,0,30,9,8
475173,507,dublin,ORIEL STREET TEST TERMINAL,"JCDecaux Ireland, 52 Oriel Street Lower, Dublin 1","{'latitude': 53.35463, 'longitude': -6.242615}",False,False,OPEN,2021-11-18 07:11:16,True,...,1,1,0,1,0,0,0,18,11,7
475284,208,bruxelles,208 - MELBA,MELBA - AV NELLIE MELBA (FACE 1 a 9) / NELLIE ...,"{'latitude': 50.832565, 'longitude': 4.294311}",False,False,CLOSED,2021-11-22 08:35:26,False,...,25,0,0,0,0,0,0,22,11,8
475432,286,bruxelles,286 - BENS,BENS - CHEE D'ALSEMBERG DEVANT LE 548 / ALSEMB...,"{'latitude': 50.807811, 'longitude': 4.336789}",False,False,CLOSED,2021-09-30 08:37:53,False,...,20,0,0,0,0,0,0,30,9,8


We can safely drop these rows. The 2 stations in Bruxelles are closed and disconnected. The station in Dublin has only 2 rows

In [8]:
bikes_data_df.drop(bikes_data_df[bikes_data_df["last_update_month"] < 12].index, axis=0,inplace=True)

Creating a multilevel index using number, contractName, and lastUpdate

In [9]:
bikes_data_df = bikes_data_df.set_index(["number", "contractName", "lastUpdate"]).sort_index()

Now, the dataframe is ready for analysis

### Answering the questions (Analysis Part)

Let's start by looking at the average number of updates for each station before Christmas

In [10]:
avg_number_of_updates_before_christmas = round(bikes_data_df[bikes_data_df.last_update_day < 25].groupby(["number", "contractName"])\
                       .size().mean())
                        

avg_number_of_updates_after_christmas = round(bikes_data_df[bikes_data_df.last_update_day > 25].groupby(["number", "contractName"])\
                       .size().mean())

print(f"Average number of updates before Christmas: {avg_number_of_updates_before_christmas}\nAverage number of updates after Christmas: {avg_number_of_updates_after_christmas}")

Average number of updates before Christmas: 584
Average number of updates after Christmas: 522


There seems to be a slight drop in activity post-Christmas. This may be because most businesses around the world follow a relaxed schedule from Christmas through 31 December 

Next we look at for pre- and post-christmas, what are the top 10 stations that have higher than average number of updates. We believe that both list will be almost the same

In [11]:
# top 10 higher than average updates before christmas

# first we need to get the updates that were pre-christmas and group them by station name and number

top_10_pre_christmas = bikes_data_df[bikes_data_df.last_update_day < 25].groupby(["number", "contractName"])

# then we get count of updates for each station

top_10_pre_christmas = top_10_pre_christmas.size()

# filter out those that have count of updates above average

top_10_pre_christmas = top_10_pre_christmas[top_10_pre_christmas > avg_number_of_updates_before_christmas]

top_10_pre_christmas = top_10_pre_christmas.sort_values(ascending=False).head(10)



# top 10 higher than average updates after christmas

# first we need to get the updates that were post-christmas and group them by station name and number

top_10_post_christmas = bikes_data_df[bikes_data_df.last_update_day > 25].groupby(["number", "contractName"])

# then we get count of updates for each station

top_10_post_christmas = top_10_post_christmas.size()

# filter out those that have count of updates above average

top_10_post_christmas = top_10_post_christmas[top_10_post_christmas > avg_number_of_updates_after_christmas]

top_10_post_christmas = top_10_post_christmas.sort_values(ascending=False).head(10)

In [12]:
print(top_10_pre_christmas)
print(top_10_post_christmas)

number  contractName
5       bruxelles       4153
205     bruxelles       3561
119     bruxelles       2275
3011    lyon            1891
105     bruxelles       1771
224     bruxelles       1736
215     bruxelles       1723
3003    lyon            1392
1002    lyon            1237
38      bruxelles       1149
dtype: int64
number  contractName
5       bruxelles       1877
224     bruxelles       1783
215     bruxelles       1722
2       bruxelles       1677
70      bruxelles       1409
3011    lyon            1189
38      bruxelles       1037
187     bruxelles       1024
180     bruxelles       1015
105     bruxelles        976
dtype: int64


It is interesting to note that both lists look somewhat different. Also, post-christmas, the number of updates have reduced significantly

#### 1. If we compare the 4 cities, which city has the most active bike ridership and which city has the least active one?

To answer this, we look at the standard deviation of totalStands_availabilities.bikes grouped by contractName

In [13]:
# grouping the data by cities

city_grouping = bikes_data_df.groupby("contractName")

# calculating mean and standard deviation of total available bikes

city_grouping["totalStands_availabilities.bikes"].agg(["std", "mean"])

,std,mean
contractName,,
bruxelles,4.344886,11.330077
dublin,7.501828,12.864938
lyon,7.219247,9.861371
toyama,3.628555,9.480609


Going just by standard deviation, we can simply conclude that Dublin has the most active ridership. However, we may be wrong because each city has a different scale (range) for totalStands_availabilities.bikes attribute. Therefore, for each city, we instead look at the normalized standard deviation which is standard deviation divided by the mean

In [14]:
# user-defined function that can be used within aggs

def normal_std(group):
  try:
    return group.std() / group.mean()
  except ZeroDivisionError:
        return 0

In [15]:
city_grouping["totalStands_availabilities.bikes"]\
.agg([("standard deviation","std"), "mean", ("normalized standard deviation", normal_std)]).T

contractName,bruxelles,dublin,lyon,toyama
standard deviation,4.344886,7.501828,7.219247,3.628555
mean,11.330077,12.864938,9.861371,9.480609
normalized standard deviation,0.383482,0.583122,0.732073,0.382734


From the normalized standard deviation, it is clear that Lyon, France has the most active ridership and Toyama, Japan has the least (relatively speaking), although Toyama and Bruxelles are almost equal.

Standard deviation is a measure of variability. The more the number of available bikes vary, the more active is the ridership

We could have used the variability in the number of available stands. However, some stations might have stands that are forever empty and this fact affects the standard deviation metric. Therefore, variability in the number of bikes available is a much more appropriate measure.

#### 2. Which city has the maximum total stands capacity across all its stations? Total capacity is sum of main stand capacity and overflow stand capacity.

We first use the city_grouping group by dataframe created before to sum the capacity of all stations grouped by their cities

In [16]:
city_stations_capacity = city_grouping["totalStands_capacity"].transform(lambda x: sum(set(x)))

What we just created is a series that has the total station capacities per city. However, this value is being repeated. So, we use drop_duplicates, convert series to dataframe and transpose it

In [17]:
city_stations_capacity.drop_duplicates().droplevel([0,2]).to_frame().T

contractName,bruxelles,toyama,dublin,lyon
totalStands_capacity,724,260,482,1320


Lyon has the most total capacities across all its stations. This could explain the high active ridership of Lyon. Because there are more stations in Lyon, there are more riders. Also, intrestingly, although Bruxelles and Toyama have almost the same ridership (as we observed from the previous analysis), Bruxelles has almost 3 times the total capacity of Toyama. Quite possibly, most of the stands in Bruxelles are just empty and are never used to park a bike

#### 3. Pre-christmas, on a daily basis, which are the top 10 stations having the most active ridership?

In [18]:
# we first slice the dataset to get data for pre-christmas

pre_christmas_daily = bikes_data_df[bikes_data_df.last_update_day < 25]

In [19]:
# next we create a pivot table that aggregates over days, number, and contractName. 
# We use normalized standard deviation that we created earlier as aggfunction

pre_christmas_daily_pivot = pd.pivot_table(pre_christmas_daily, 
               index=["number", "contractName"], 
               columns=["last_update_day"], 
               values=["totalStands_availabilities.bikes"], aggfunc=[normal_std])

C:\Users\dines\AppData\Local\Temp\ipykernel_19040\116835333.py:5: RuntimeWarning: invalid value encountered in double_scalars
  return group.std() / group.mean()


For each day, we need to sort the values individually because each day is a column and sorting one column unsorts the other

In [20]:
# sorting 19 Dec values

print(pre_christmas_daily_pivot[("normal_std", "totalStands_availabilities.bikes", 19)].sort_values(ascending=False).head(10))

# sorting 20 Dec values

print(pre_christmas_daily_pivot[("normal_std", "totalStands_availabilities.bikes", 20)].sort_values(ascending=False).head(10))

# sorting 21 Dec values

print(pre_christmas_daily_pivot[("normal_std", "totalStands_availabilities.bikes", 21)].sort_values(ascending=False).head(10))

# sorting 22 Dec values

print(pre_christmas_daily_pivot[("normal_std", "totalStands_availabilities.bikes", 22)].sort_values(ascending=False).head(10))

number  contractName
4021    lyon            5.196152
5016    lyon            2.823709
19001   lyon            2.823709
3011    lyon            2.481496
4008    lyon            2.391652
4006    lyon            2.391652
4002    lyon            2.137576
3080    lyon            1.686662
8002    lyon            1.610153
10113   lyon            1.566717
Name: (normal_std, totalStands_availabilities.bikes, 19), dtype: float64
number  contractName
79      bruxelles       2.157104
3008    lyon            1.565572
4022    lyon            1.485868
192     bruxelles       1.388184
4005    lyon            1.331715
1024    lyon            1.321775
555     lyon            1.281327
1003    lyon            1.279883
7051    lyon            1.251076
5026    lyon            1.236065
Name: (normal_std, totalStands_availabilities.bikes, 20), dtype: float64
number  contractName
25      dublin          1.526518
20002   lyon            1.332911
3008    lyon            1.257534
3009    lyon            1.212829

From the 4 dataframes, we can conclude that Lyon has the most active stations on a daily basis.

#### 4. Pre-christmas, on an hourly basis, which are the top 10 stations having the most active ridership?

To answer this question, we first need to create groupings by number, contractName, and last_update_hour and calculate the aggregated normal standard deviation. We performing said grouping on the pre_cristmas_daily dataframe defined earlier

In [21]:
top_10_hourly_group = pre_christmas_daily\
                      .groupby(["number", "contractName", "last_update_hour"])["totalStands_availabilities.bikes"]\
                      .agg([("bikes_normal_std", normal_std)])\
                      .reset_index()

C:\Users\dines\AppData\Local\Temp\ipykernel_19040\116835333.py:5: RuntimeWarning: invalid value encountered in double_scalars
  return group.std() / group.mean()


In [22]:
top_10_hourly_group\
.sort_values(["last_update_hour", "bikes_normal_std"], ascending=[True, False])\
.groupby("last_update_hour").head(10)

,number,contractName,last_update_hour,bikes_normal_std
12037,2001,lyon,0,2.051957
13045,3008,lyon,0,1.463919
4056,79,bruxelles,0,1.455214
12469,2019,lyon,0,1.455214
13621,3058,lyon,0,1.455214
14053,3090,lyon,0,1.455214
16765,7026,lyon,0,1.455214
12973,3005,lyon,0,1.452025
12997,3006,lyon,0,1.404358
13333,3031,lyon,0,1.395678


According to the output dataframe, one an hourly basis, for the top 10 most active stations pre-christmas, a vast majority are in Lyon. Some are in Bruxelles and less so in Dublin. This proves again that the bike stations in Toyama are relatively the least active.

#### 5. Do the cities in the lists from 3 and 4 change significantly post-christmas?

In [23]:
# we first slice the dataset to get data for post-christmas

post_christmas_daily = bikes_data_df[bikes_data_df.last_update_day > 25]

In [24]:
# next we create a pivot table that aggregates over days, number, and contractName. 
# We use normalized standard deviation that we created earlier as aggfunction

post_christmas_daily_pivot = pd.pivot_table(post_christmas_daily, 
               index=["number", "contractName"], 
               columns=["last_update_day"], 
               values=["totalStands_availabilities.bikes"], aggfunc=[normal_std])

C:\Users\dines\AppData\Local\Temp\ipykernel_19040\116835333.py:5: RuntimeWarning: invalid value encountered in double_scalars
  return group.std() / group.mean()


For each day, we need to sort the values individually because each day is a column and sorting one column unsorts the other

In [25]:
# sorting 26 Dec values

print(post_christmas_daily_pivot[("normal_std", "totalStands_availabilities.bikes", 26)].sort_values(ascending=False).head(10))

# sorting 27 Dec values

print(post_christmas_daily_pivot[("normal_std", "totalStands_availabilities.bikes", 27)].sort_values(ascending=False).head(10))

# sorting 28 Dec values

print(post_christmas_daily_pivot[("normal_std", "totalStands_availabilities.bikes", 28)].sort_values(ascending=False).head(10))

# sorting 29 Dec values

print(post_christmas_daily_pivot[("normal_std", "totalStands_availabilities.bikes", 29)].sort_values(ascending=False).head(10))

number  contractName
8037    lyon            5.522133
8059    lyon            2.782086
14001   lyon            2.151404
6041    lyon            1.509405
3024    lyon            1.468548
7015    lyon            1.313489
10063   lyon            1.281551
10115   lyon            1.277152
1036    lyon            1.262137
7023    lyon            1.261426
Name: (normal_std, totalStands_availabilities.bikes, 26), dtype: float64
number  contractName
8003    lyon            1.851109
10004   lyon            1.327837
10110   lyon            1.326086
8059    lyon            1.256436
7015    lyon            1.251278
6041    lyon            1.247480
7055    lyon            1.183066
2015    lyon            1.155800
8037    lyon            1.138377
10013   lyon            1.134777
Name: (normal_std, totalStands_availabilities.bikes, 27), dtype: float64
number  contractName
62      dublin          1.265544
10019   lyon            1.083912
8021    lyon            1.080507
33      dublin          1.054279

Even post christmas, Lyon remains the city with the most active stations, on a daily basis. However, we see some Dublin stations in the list too

In [26]:
top_10_hourly_group = post_christmas_daily\
                      .groupby(["number", "contractName", "last_update_hour"])["totalStands_availabilities.bikes"]\
                      .agg([("bikes_normal_std", normal_std)])\
                      .reset_index()

top_10_hourly_group\
.sort_values(["last_update_hour", "bikes_normal_std"], ascending=[True, False])\
.groupby("last_update_hour").head(10)

C:\Users\dines\AppData\Local\Temp\ipykernel_19040\116835333.py:5: RuntimeWarning: invalid value encountered in double_scalars
  return group.std() / group.mean()


,number,contractName,last_update_hour,bikes_normal_std
13783,3080,lyon,0,1.455214
16471,7015,lyon,0,1.455214
18895,10005,lyon,0,1.455214
19039,10013,lyon,0,1.455214
13399,3038,lyon,0,1.455214
20839,12001,lyon,0,1.395678
13591,3058,lyon,0,1.358262
14239,3138,lyon,0,1.348637
13135,3015,lyon,0,1.242320
13063,3011,lyon,0,1.185565


Post-christmas, on an hourly basis, it seems that Lyon has even more stations on the top 10 most active stations

#### 6. Of the cities in the dataset that have stations with both mechanical and electric bikes, are there any cities having strong preference for either mechanical or electric bikes?

To answer this quesiton, we first need to segregate the stations that have both mechnical and electic bikes and stations that have only mechanical ones

In [27]:
mech_and_electric = bikes_data_df.groupby(["number", "contractName"]).agg({
    "totalStands_availabilities.electricalBikes": "sum"
}).reset_index()

# we filter the grouped dataframe to find stations that have 0 electric bikes and create a new dataframe from it

non_electric_stations = mech_and_electric[mech_and_electric["totalStands_availabilities.electricalBikes"] == 0]

non_electric_stations

,number,contractName,totalStands_availabilities.electricalBikes
1,1,toyama,0
4,2,toyama,0
7,3,toyama,0
10,4,toyama,0
13,5,toyama,0
16,6,toyama,0
19,7,toyama,0
22,8,toyama,0
25,9,toyama,0
28,10,toyama,0


It may be the case that none of the stations in Toyama have electric bikes. Let us verify this using the bikes_data_df dataframe

In [28]:
bikes_data_df[(bikes_data_df.index.get_level_values("contractName") == "toyama")\
              & (bikes_data_df["totalStands_availabilities.electricalBikes"] > 0)]

,,,name,address,position,banking,bonus,status,connected,overflow,shape,totalStands_capacity,...,mainStands_capacity,mainStands_availabilities.bikes,mainStands_availabilities.stands,mainStands_availabilities.mechanicalBikes,mainStands_availabilities.electricalBikes,mainStands_availabilities.electricalInternalBatteryBikes,mainStands_availabilities.electricalRemovableBatteryBikes,last_update_day,last_update_month,last_update_hour
number,contractName,lastUpdate,,,,,,,,,,,,,,,,,,,,,


So, now we can discount all stations in Toyama and the few stations in Lyon and Bruxelles completely for this particular analysis

In [29]:
# we first take columns from the non_electric_stations dataframe and convert it to a list

keys = list(non_electric_stations.columns.values)

# we need to reset the index for bikes_data_df and set the newly created keys list as the new index. 
# The same needs to be done for the non_electric_stations dataframe

idx1 = bikes_data_df.reset_index().set_index(keys).index
idx2 = non_electric_stations.set_index(keys).index

# this is where we take the orignial bikes_data_df and filter out all stations with 0 electric bikes, 
# leaving stations with both electric and mechanical bikes

mech_electric_analysis_df = bikes_data_df[~idx1.isin(idx2)]

Finally, we use the newly created mech_electric_analysis_df to obtain the normal standard deviation of both mechnical and electric bikes grouped by contractName (city)

In [30]:
mech_electric_analysis_df.groupby("contractName").agg({
    "totalStands_availabilities.electricalBikes": normal_std,
    "totalStands_availabilities.mechanicalBikes": normal_std
})

,totalStands_availabilities.electricalBikes,totalStands_availabilities.mechanicalBikes
contractName,,
bruxelles,0.636122,0.486301
dublin,0.924482,0.615470
lyon,0.912668,0.828037


Clearly, all 3 cities that have electric bikes prefer electric over mechanical bikes. For Dublin, the difference is very significant compared to the other two.

### Closing Thoughts
We can safely conclude that most stations in Lyon, France and Bruxelles, Belgium have the most active stations. Dublin, Ireland has fairly active ridership as well. Thus, it would make sense for our media client to focus their resources on the stations indicated in our analysis